In [3]:
import os
import numpy as np
from openai import OpenAI
import chromadb
from chromadb.api.types import EmbeddingFunction

# ========= Config =========
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
assert GITHUB_TOKEN, "Falta GITHUB_TOKEN"
GITHUB_ENDPOINT = "https://models.github.ai/inference"
LLM_MODEL = "openai/gpt-4.1"                 # Chat/LLM
EMB_MODEL = "openai/text-embedding-3-small"  # Embeddings

# ========= Clientes =========
llm_client = OpenAI(base_url=GITHUB_ENDPOINT, api_key=GITHUB_TOKEN)

# 👉 Deja que el token decida tenant/database
chroma = chromadb.CloudClient(api_key=os.getenv("CHROMADB_TOKEN"))

# ========= Wrapper de Embeddings para Chroma (GitHub Models) =========
class GithubOpenAIEmbeddingFunction(EmbeddingFunction[str]):
    def __init__(self, client: OpenAI, model: str):
        self.client = client
        self.model = model

    def __call__(self, inputs):
        if isinstance(inputs, str):
            inputs = [inputs]
        resp = self.client.embeddings.create(model=self.model, input=inputs)
        return [np.array(item.embedding, dtype=np.float32) for item in resp.data]

emb_fn = GithubOpenAIEmbeddingFunction(llm_client, EMB_MODEL)

In [4]:
# =========  LLM =========
chat = llm_client.chat.completions.create(
    model=LLM_MODEL,
    messages=[
        {"role": "system", "content": "Eres un asistente útil y conciso."},
        {"role": "user", "content": "¿Cuál es la capital de Francia?"}
    ]
)

In [ ]:
collection_name = "document_qa_collection"

collection = chroma.get_or_create_collection(
    name=collection_name,
    embedding_function=emb_fn, 
)

print("Colección lista:", collection.name)


Colección lista: document_qa_collection


La capital de Francia es París.


NameError: name 'messages' is not defined